In [203]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/PratikBorkar04/consignment_pricing/main/notebook/dataset/CatBoosting_dataset.csv")

In [259]:
df["freight_cost"]

0        3521
1       15007
2       44450
3        4920
4        5212
        ...  
6248     9181
6249     9181
6250     9181
6251     9181
6252     9181
Name: freight_cost, Length: 6253, dtype: int64

In [260]:
df.head()


,country,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,freight_cost,location,distance
0,Vietnam,EXW,Air,240,1000,6200,6,0.03,358,3521,India,3157
1,Vietnam,EXW,Air,60,31920,127360,3,0.07,1855,15007,India,3157
2,Vietnam,EXW,Air,60,38000,121600,3,0.05,7590,44450,India,3157
3,Nigeria,EXW,Air,240,416,2225,5,0.02,504,4920,India,7633
4,Tanzania,EXW,Air,60,16667,60834,3,0.06,1478,5212,India,5659


In [261]:
df.isnull().sum()

country               0
vendor                0
shipment_mode         0
unit_of_measure       0
line_item_quantity    0
line_item_value       0
pack_price            0
unit_price            0
weight                0
freight_cost          0
location              0
distance              0
dtype: int64

In [262]:
df.shape

(6253, 12)

In [263]:
from sklearn import preprocessing
  
# label_encoder object knows 

label_encoder = preprocessing.LabelEncoder()
categorical_columns = ['country', 'vendor', 'shipment_mode', 'location']
for i in categorical_columns:
    print(df[i].unique())
    df[i]= label_encoder.fit_transform(df[i])
    # Encode labels in column 'species'.
    print(df[i].unique())
    print("--"*18)


['Vietnam' 'Nigeria' 'Tanzania' 'Zambia' 'Rwanda' 'Zimbabwe'
 "Côte d'Ivoire" 'Haiti' 'South Africa' 'Uganda' 'Mozambique' 'Congo, DRC']
[ 9  4  7 10  5 11  1  2  6  8  3  0]
------------------------------------
['EXW' 'CIP' 'FCA' 'DDP' 'RDC' 'DAP']
[3 0 4 2 5 1]
------------------------------------
['Air' 'Truck' 'Air Charter' 'Ocean']
[0 3 1 2]
------------------------------------
['India' 'Germany' 'France' 'Thailand' ' Nashik' 'Switzerland' ' Haarlem'
 ' Port Elizabeth' 'Canada' 'South Africa' 'Puerto Rico' ' Latina'
 'Poland' 'Cyprus' 'China']
[ 9  8  7 14  2 13  0  3  4 12 11  1 10  6  5]
------------------------------------


In [264]:
df.head(1)

,country,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,freight_cost,location,distance
0,9,3,0,240,1000,6200,6,0.03,358,3521,9,3157


In [265]:
import pandas as pd

# Assuming your DataFrame is called 'df'
columns = ['line_item_quantity', 'line_item_value', 'freight_cost']
n_max = 10
n_min = 5

# Get the 10 maximum values from each column
max_values = df.nlargest(n_max, columns)

# Get the 5 minimum values from each column
min_values = df.nsmallest(n_min, columns)

# Print the maximum values
print("Maximum Values:")
print(max_values)

# Print the minimum values
print("Minimum Values:")
print(min_values)


Maximum Values:
      country  vendor  shipment_mode  unit_of_measure  line_item_quantity   
5883       10       5              3               30              600906  \
5892       10       5              3               30              555197   
3413        6       2              2               60              515000   
2122        6       2              2               60              515000   
2493        6       2              2               60              515000   
4130        3       5              3               30              460041   
3430        6       2              2               60              440000   
4502       11       5              3               30              400000   
1941        6       2              2               60              356876   
3833        4       5              1               60              350505   

      line_item_value  pack_price  unit_price  weight  freight_cost  location   
5883          5768697           9        0.32   67880  

In [266]:
#Preapare X and y variables
X = df.drop(columns=['freight_cost'],axis=1)    

In [267]:
X.head()

,country,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,location,distance
0,9,3,0,240,1000,6200,6,0.03,358,9,3157
1,9,3,0,60,31920,127360,3,0.07,1855,9,3157
2,9,3,0,60,38000,121600,3,0.05,7590,9,3157
3,4,3,0,240,416,2225,5,0.02,504,9,7633
4,7,3,0,60,16667,60834,3,0.06,1478,9,5659


In [268]:
y = df['freight_cost']


In [269]:
y

0        3521
1       15007
2       44450
3        4920
4        5212
        ...  
6248     9181
6249     9181
6250     9181
6251     9181
6252     9181
Name: freight_cost, Length: 6253, dtype: int64

In [270]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


((5002, 11), (1251, 11))

In [271]:
print(X_train)
print(X_test)

      country  vendor  shipment_mode  unit_of_measure  line_item_quantity   
478         6       2              0               60                1000  \
5099        4       5              1              120               43169   
1203        4       5              0               30               13473   
5645        1       5              0              240                3500   
142         4       3              0               60                4632   
...       ...     ...            ...              ...                 ...   
3772        1       5              3               60               40035   
5191        4       5              1               60                 912   
5226        1       5              0               30                6464   
5390        4       5              0               90                6654   
860         4       5              0               60                 451   

      line_item_value  pack_price  unit_price  weight  location  distance  

In [272]:
#Create an Evaluate Function to give all metrics after model Training
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [273]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 9098.3978
- Mean Absolute Error: 4452.0787
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.9721
- Mean Absolute Error: 4658.0867
- R2 Score: 0.1082


Lasso
Model performance for Training set
- Root Mean Squared Error: 9098.4010
- Mean Absolute Error: 4451.8990
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.7062
- Mean Absolute Error: 4658.0058
- R2 Score: 0.1083


Ridge
Model performance for Training set
- Root Mean Squared Error: 9098.3978
- Mean Absolute Error: 4452.0492
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.9620
- Mean Absolute Error: 4658.0614
- R2 Score: 0.1082


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 8113.6404
- Mean Absolute Error: 3589.38

In [274]:
#Results
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
7,CatBoosting Regressor,0.608919
5,Random Forest Regressor,0.506735
6,XGBRegressor,0.455895
4,Decision Tree,0.287463
3,K-Neighbors Regressor,0.155706
1,Lasso,0.108251
2,Ridge,0.108209
0,Linear Regression,0.108207
8,AdaBoost Regressor,-0.473243


In [275]:
#catBoostRegressor
CatBoosting_Regressor = CatBoostRegressor(verbose=False)
CatBoosting_Regressor = CatBoosting_Regressor.fit(X_train, y_train)
y_pred = CatBoosting_Regressor.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 60.89


In [276]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df


,Actual Value,Predicted Value,Difference
1703,6292,12181.001846,-5889.001846
5449,9181,8077.588173,1103.411827
5058,5933,4097.977158,1835.022842
1149,651,2736.243698,-2085.243698
432,9181,9047.200177,133.799823
...,...,...,...
416,9181,9358.023535,-177.023535
6111,20520,12897.164194,7622.835806
3185,721,-161.521385,882.521385
2025,4726,5793.829780,-1067.829780


In [5]:
df["unit_price"].max

<bound method NDFrame._add_numeric_operations.<locals>.max of 0       0.03
1       0.07
2       0.05
3       0.02
4       0.06
        ... 
6248    0.10
6249    0.06
6250    0.06
6251    0.11
6252    0.03
Name: unit_price, Length: 6253, dtype: float64>

In [278]:
import pickle
pickle_model = pickle.load(open("model copy.pkl","rb"))

In [284]:
pickle_model.predict([[9,3,0,60,31920,127360,3,0.05,1855,9,3157]])

array([8995.90452326])